In [8]:


# make local editable packages automatically reload
%load_ext autoreload
%autoreload 2

# Import dependencies
import numpy as np

# This checks to see if you have set up your GPU properly.
# CPU performance is a lot slower, but not a problem if you 
# are only processing a few images.
use_GPU = False #core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)

# for plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi'] = 300
plt.style.use('dark_background')
%matplotlib inline

import tifffile
from skimage.measure import regionprops_table
from skimage.morphology import binary_dilation
from skimage.transform import rotate
import pandas as pd
from scipy.signal import find_peaks


2023-05-11 02:09:29,800 [INFO] OpenGL_accelerate module loaded
2023-05-11 02:09:29,803 [INFO] Using accelerated ArrayDatatype
>>> GPU activated? 0


In [12]:
from tqdm.auto import tqdm
from joblib import Parallel, delayed

In [31]:
masks = tifffile.imread("segmentation_output/masks.tif")
image = tifffile.imread("segmentation_output/image.tif")
mask_labels = np.unique(masks)
def n_dilate(bin_image, n):
    for _ in range(n):
        bin_image = binary_dilation(bin_image)
    return bin_image

def smallestbox(a, b):
    r = a.any(1)
    if r.any():
        m,n = a.shape
        c = a.any(0)
        out = a[r.argmax():m-r[::-1].argmax(), c.argmax():n-c[::-1].argmax()]
        b  = b[:,r.argmax():m-r[::-1].argmax(), c.argmax():n-c[::-1].argmax()]
    else:
        out = np.empty((0,0),dtype=bool)
    return out, b

def get_interpeak(z, mask_label):
    b1, b2 = smallestbox(n_dilate(masks == mask_label,10), image)
    _ = smallestbox((masks == mask_label), image)[0]
    orientation = pd.DataFrame(regionprops_table(_*1, properties=["orientation"]))["orientation"][0]
    b1, b2 = rotate(b1, -orientation * 180/np.pi + 90), np.array([rotate(_, -orientation * 180/np.pi + 90) for _ in b2])
    centroid = regionprops_table(b1*1, properties=["centroid"])
    width = int(b2.shape[1]/10)
    
    
    centreline1 = b2[1,:,int(centroid["centroid-1"])-width:int(centroid["centroid-1"])+width].mean(axis=1)
    centreline2 = b2[3,:,int(centroid["centroid-1"])-width:int(centroid["centroid-1"])+width].mean(axis=1)
    
    
    centreline1_peaks = find_peaks(centreline1, prominence=0.006)[0]
    centreline2_peaks = find_peaks(centreline2, prominence=0.006)[0]

    if (len(centreline1_peaks) == 2) and (len(centreline2_peaks) == 2):
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
        ax1.plot(centreline1,c="r")
        ax1.scatter(centreline1_peaks, centreline1[centreline1_peaks], c = "r")
        ax1.plot(centreline2,c="b")
        ax1.scatter(centreline2_peaks, centreline2[centreline2_peaks], c = "b")
        #ax1.legend(["red", "blue"])
        ax2.imshow(b2[1])
        ax3.imshow(b2[3])
        plt.tight_layout()
        plt.savefig(f"plots/{str(z).zfill(4)}.png")
        plt.close()
        #print("Red interpeak distance:", np.diff(centreline1_peaks))
        #print("Blue interpeak distance:", np.diff(centreline2_peaks))
        return [z, np.diff(centreline1_peaks)[0], np.diff(centreline2_peaks)[0]]
    

In [22]:
interpeak_distances = Parallel(n_jobs=-1)(delayed(get_interpeak)(z, mask_label) for z, mask_label in enumerate(tqdm(mask_labels[1:])))

  0%|          | 0/822 [00:00<?, ?it/s]

In [23]:
interpeak_distances = pd.DataFrame([x for x in interpeak_distances if x is not None])
interpeak_distances.columns = ["idx", "RADA", "HADA"]
bad_idxs = [18, 34, 40, 55, 142, 150, 168, 177, 181, 185, 210, 222, 258, 264, 293, 301, 304, 307, 328, 355, 371, 378, 390, 391, 396, 427, 447, 472, 480, 481, 494, 501, 521, 524, 527, 542, 585, 601, 606, 609, 611, 615, 645, 647, 700, 708, 711, 714, 715, 747, 772]
interpeak_distances = interpeak_distances[~interpeak_distances["idx"].isin(bad_idxs)]

In [24]:
interpeak_distances

,idx,RADA,HADA
1,20,8,10
2,27,14,18
3,29,14,17
5,39,15,18
7,41,16,18
...,...,...,...
199,800,11,14
200,801,10,11
201,805,8,11
202,812,8,10


In [25]:
interpeak_distances["Average width"] = interpeak_distances[["RADA", "HADA"]].mean(axis=1)

In [26]:
interpeak_distances["RADA - HADA"] = interpeak_distances["RADA"] - interpeak_distances["HADA"]

In [30]:
interpeak_distances.to_pickle("output.pickle")